# Supply Chain Dataset

Aquí se presenta un conjunto de datos recopilado de una startup de moda y belleza. El conjunto de datos se basa en la cadena de suministro de productos de maquillaje. A continuación, se muestran todas las características del conjunto de datos:

Product Type

SKU

Price

Availability

Number of products sold

Revenue generated

Customer demographics

Stock levels

Lead times

Order quantities

Shipping times

Shipping carriers

Shipping costs

Supplier name

Location

Lead time

Production volumes

Manufacturing lead time

Manufacturing costs

Inspection results

Defect rates

Transportation modes

Routes

Costs



**Lista de Preguntas SQL:**

¿Cuál es el producto más vendido (por número de productos vendidos)?

¿Cuál es el ingreso promedio generado por tipo de producto?

¿Cuáles son los 5 productos con mayores tasas de defectos?

¿Qué proveedores tienen el mayor tiempo de entrega (Lead time)?

¿Cuál es el costo total de envío por transportadora?

¿Cuál es el costo promedio de manufactura por tipo de producto?

¿Qué productos tienen baja disponibilidad pero alta demanda (ventas superiores al promedio)?

¿Qué rutas de transporte tienen los mayores costos?

¿Qué porcentaje de productos por tipo han fallado en la inspección?

¿Qué productos tienen el mayor volumen de producción y menor costo de manufactura?

In [2]:
import pandas as pd
import sqlite3

from google.colab import files

df = pd.read_csv("supply_chain_data.csv")

conn = sqlite3.connect(":memory:")
df.to_sql("supply_chain", conn, index=False, if_exists="replace")

queries = [
    ("1. Producto más vendido (por número de productos vendidos)",
     """
     SELECT "Product Type", SUM("Number of products sold") AS Total_Sold
     FROM supply_chain
     GROUP BY "Product Type"
     ORDER BY Total_Sold DESC
     LIMIT 1;
     """),

    ("2. Ingreso promedio generado por tipo de producto",
     """
     SELECT "Product Type", AVG("Revenue generated") AS Avg_Revenue
     FROM supply_chain
     GROUP BY "Product Type"
     ORDER BY Avg_Revenue DESC;
     """),

    ("3. Top 5 productos con mayores tasas de defectos",
     """
     SELECT SKU, "Defect rates"
     FROM supply_chain
     ORDER BY "Defect rates" DESC
     LIMIT 5;
     """),

    ("4. Proveedores con mayor tiempo de entrega (Lead time)",
     """
     SELECT "Supplier name", MAX("Lead time") AS Max_Lead_Time
     FROM supply_chain
     GROUP BY "Supplier name"
     ORDER BY Max_Lead_Time DESC
     LIMIT 5;
     """),

    ("5. Costo total de envío por transportadora",
     """
     SELECT "Shipping carriers", SUM("Shipping costs") AS Total_Shipping_Cost
     FROM supply_chain
     GROUP BY "Shipping carriers"
     ORDER BY Total_Shipping_Cost DESC;
     """),

    ("6. Costo promedio de manufactura por tipo de producto",
     """
     SELECT "Product Type", AVG("Manufacturing costs") AS Avg_Manufacturing_Cost
     FROM supply_chain
     GROUP BY "Product Type"
     ORDER BY Avg_Manufacturing_Cost DESC;
     """),

    ("7. Productos con baja disponibilidad y alta demanda",
     """
     SELECT SKU, "Availability", "Number of products sold"
     FROM supply_chain
     WHERE "Availability" < 10
       AND "Number of products sold" > (
         SELECT AVG("Number of products sold") FROM supply_chain
     );
     """),

    ("8. Rutas de transporte con mayores costos",
     """
     SELECT "Routes", SUM("Costs") AS Total_Cost
     FROM supply_chain
     GROUP BY "Routes"
     ORDER BY Total_Cost DESC
     LIMIT 5;
     """),

    ("9. Porcentaje de productos fallados por tipo",
     """
     SELECT "Product Type",
            ROUND(100.0 * SUM(CASE WHEN "Inspection results" = 'Failed' THEN 1 ELSE 0 END) / COUNT(*), 2) AS Failure_Percentage
     FROM supply_chain
     GROUP BY "Product Type"
     ORDER BY Failure_Percentage DESC;
     """),

    ("10. Productos con mayor producción y menor costo de manufactura",
     """
     SELECT SKU, "Production volumes", "Manufacturing costs"
     FROM supply_chain
     ORDER BY "Production volumes" DESC, "Manufacturing costs" ASC
     LIMIT 10;
     """)
]

# Paso 6: Ejecutar las consultas y mostrar los resultados
for title, sql in queries:
    print(f"\n--- {title} ---")
    result = pd.read_sql_query(sql, conn)
    display(result)



--- 1. Producto más vendido (por número de productos vendidos) ---


,Product type,Total_Sold
0,skincare,20731



--- 2. Ingreso promedio generado por tipo de producto ---


,Product type,Avg_Revenue
0,cosmetics,6212.356385
1,skincare,6040.704053
2,haircare,5131.040900



--- 3. Top 5 productos con mayores tasas de defectos ---


...

,Routes,Total_Cost
0,Route B,22039.384026
1,Route A,20875.774494
2,Route C,10009.419696



--- 9. Porcentaje de productos fallados por tipo ---


,Product type,Failure_Percentage
0,skincare,0.0
1,haircare,0.0
2,cosmetics,0.0



--- 10. Productos con mayor producción y menor costo de manufactura ---


,SKU,Production volumes,Manufacturing costs
0,SKU76,985,64.323598
1,SKU2,971,30.688019
2,SKU47,964,19.712993
3,SKU9,963,47.957602
4,SKU82,955,4.465278
5,SKU88,953,68.184919
6,SKU3,937,35.624741
7,SKU25,934,78.280383
8,SKU60,929,87.213058
9,SKU99,921,38.072899
